In [6]:
import pandas as pd
import numpy as np
import requests as req
import datetime
import json
import os

In [7]:
def max0(sr):
     return (sr != 0).cumsum().value_counts().max() - (0 if (sr != 0).cumsum().value_counts().idxmax()==0 else 1)

def string_2_date(string):
    return datetime.datetime.strptime(string, '%Y-%m-%d')

def swap_mask(df):
    try:
        mask = ((df['t_kWh']>0.3).any()) or ((df['t_kVAh']>0.3).any())
    except:
        print(df['t_kWh'].dtype)
        print(df['t_kVAh'].dtype)
        print('Hoola')
        print(df['y_PF Total'].
              dtype)
    return mask

def swap_num_days(df):
    swapped = 0
    g = df.groupby(df.index.date)
    for n,gr in g :
        if swap_mask(gr)== True:
            swapped = swapped+1
    return swapped

def get_data(start_date,end_date,meter_list):
    df_list = []
    base_path = "http://www.sumeruservices.com/PrepayService/rest/RawData/loadProfileDatasetSPCeew/Society/CEEW"
    url_key = "http://www.sumeruservices.com/PrepayService/rest/authentication/AuthenticationTest/ceew_1/sumeru@9711"
    
    issue = []
    
    for meter in meter_list:
        key = req.get(url_key)
        new_key = json.loads(key.text)
        new_key = new_key['apiKey']
        new_key = '2'+new_key+'4'
        print('fetching data for', meter)
        url = base_path+'/'+meter+'/'+start_date+'/'+end_date+'/'+new_key
        print(url)
        try:
            content = json.loads(req.get(url).text)
            df = pd.DataFrame(content['data'])
            print(df.shape[0])
            #renaming
    #         if meter == 'MH04':
    #             meter = 'MH38'


            df['meter'] = [meter]*df.shape[0]
    #         df.to_csv('data/'+meter+'.csv')
            df_list.append(df)
            print('Fetched data')
        except:
            
            issue.append(meter)
            print('Could not fetch for', meter)
    final_df =pd.concat([df_list[i] for i in range(len(df_list))],axis=0)
    return final_df

def get_meter_list():
    mh_removed = [5, 13, 19, 32, 40, 44, 16, 29, 4]
    br_removed = [1, 21, 25, 40, 47, 41]
    mh_num = [i for i in range(1,48)]
    br_num = [i for i in range(1,53)]
    mh_meters = ['MH'+str(item).zfill(2) for item in mh_num]
    br_meters = ['BR'+str(item).zfill(2) for item in br_num]
    for i in mh_removed:
        mh_meters.remove('MH'+str(i).zfill(2))
    for i in br_removed:
        br_meters.remove('BR'+str(i).zfill(2))
    meter_list = [mh_meters,br_meters] 
    final_meter_list = [item for sublist in meter_list for item in sublist]
    return final_meter_list

In [8]:
# Modified


def max0(sr):
     return (sr != 0).cumsum().value_counts().max() - (0 if (sr != 0).cumsum().value_counts().idxmax()==0 else 1)

def string_2_date(string):
    return datetime.datetime.strptime(string, '%Y-%m-%d')

def swap_mask(df):
    try:
        mask = ((df['t_kWh']>0.3).any()) or ((df['t_kVAh']>0.3).any())
    except:
        print(df['t_kWh'].dtype)
        print(df['t_kVAh'].dtype)
        print('Hoola')
        print(df['y_PF Total'].
              dtype)
    return mask

def swap_num_days(df):
    swapped = 0
    g = df.groupby(df.index.date)
    
    # power cut days
    d = {k:0 for k in ['8-12','12-20','20-24','24']}
    
    
    for n,gr in g :
        if swap_mask(gr)== True:
            swapped = swapped+1
        
        # power cut hours
        zero = gr.loc[gr['z_Avg Voltage (Volt)']==0]
        length = zero.shape[0]
        if 160<length<=240:
            d['8-12'] = d['8-12'] +1
        elif 240<length<=400:
            d['12-20'] = d['12-20'] +1
        elif 400<length<480:
            d['20-24'] = d['20-24']+1
        elif length==480:
            d['24']= d['24']+1
        
        
    return swapped,d

def get_data(start_date,end_date,meter_list):
    df_list = []
    base_path = "http://www.sumeruservices.com/PrepayService/rest/RawData/loadProfileDatasetSPCeew/Society/CEEW"
    url_key = "http://www.sumeruservices.com/PrepayService/rest/authentication/AuthenticationTest/ceew_1/sumeru@9711"
    
    issue = []
    
    for meter in meter_list:
        key = req.get(url_key)
        new_key = json.loads(key.text)
        new_key = new_key['apiKey']
        new_key = '2'+new_key+'4'
        print('fetching data for', meter)
        url = base_path+'/'+meter+'/'+start_date+'/'+end_date+'/'+new_key
        print(url)
        try:
            content = json.loads(req.get(url).text)
            df = pd.DataFrame(content['data'])
            print(df.shape[0])
            #renaming
    #         if meter == 'MH04':
    #             meter = 'MH38'


            df['meter'] = [meter]*df.shape[0]
    #         df.to_csv('data/'+meter+'.csv')
            df_list.append(df)
            print('Fetched data')
        except:
            issue.append(meter)
            print('Could not fetch for', meter)
    final_df =pd.concat([df_list[i] for i in range(len(df_list))],axis=0)
    return final_df

def get_meter_list():
    mh_removed = [5, 13, 19, 32, 40, 44, 16, 29, 4]
    br_removed = [1, 21, 25, 40, 47, 41]
    mh_num = [i for i in range(1,48)]
    br_num = [i for i in range(1,53)]
    mh_meters = ['MH'+str(item).zfill(2) for item in mh_num]
    br_meters = ['BR'+str(item).zfill(2) for item in br_num]
    for i in mh_removed:
        mh_meters.remove('MH'+str(i).zfill(2))
    for i in br_removed:
        br_meters.remove('BR'+str(i).zfill(2))
    meter_list = [mh_meters,br_meters] 
    final_meter_list = [item for sublist in meter_list for item in sublist]
    return final_meter_list

In [4]:
meters = get_meter_list()
# len(meters)

In [9]:
met_num = ['11','13','24','28','29','31','30','27','49','50','19','16','22','05','06','08','04','39','34','33','36','35','37','32','43','44','45','48','15','51']
for i in range(len(met_num)):
    met_num[i] = 'BR' + met_num[i]

In [10]:
meters = sorted(met_num)



In [11]:
meters

['BR04',
 'BR05',
 'BR06',
 'BR08',
 'BR11',
 'BR13',
 'BR15',
 'BR16',
 'BR19',
 'BR22',
 'BR24',
 'BR27',
 'BR28',
 'BR29',
 'BR30',
 'BR31',
 'BR32',
 'BR33',
 'BR34',
 'BR35',
 'BR36',
 'BR37',
 'BR39',
 'BR43',
 'BR44',
 'BR45',
 'BR48',
 'BR49',
 'BR50',
 'BR51']

In [8]:
end_date = str(datetime.date.today())
start_date = '2021-03-01'
end_date = '2021-07-14'
df = get_data(start_date,end_date,meters)

fetching data for BR04
http://www.sumeruservices.com/PrepayService/rest/RawData/loadProfileDatasetSPCeew/Society/CEEW/BR04/2021-03-01/2021-07-14/25375636162624305263136573734
56050
Fetched data
fetching data for BR05
http://www.sumeruservices.com/PrepayService/rest/RawData/loadProfileDatasetSPCeew/Society/CEEW/BR05/2021-03-01/2021-07-14/25375636162624305691336573734
27730
Fetched data
fetching data for BR06
http://www.sumeruservices.com/PrepayService/rest/RawData/loadProfileDatasetSPCeew/Society/CEEW/BR06/2021-03-01/2021-07-14/25375636162624305933236573734
64210
Fetched data
fetching data for BR08
http://www.sumeruservices.com/PrepayService/rest/RawData/loadProfileDatasetSPCeew/Society/CEEW/BR08/2021-03-01/2021-07-14/25375636162624306379636573734
0
Fetched data
fetching data for BR11
http://www.sumeruservices.com/PrepayService/rest/RawData/loadProfileDatasetSPCeew/Society/CEEW/BR11/2021-03-01/2021-07-14/25375636162624306509836573734
64210
Fetched data
fetching data for BR13
http://www.

In [9]:
# end_date = str(datetime.date.today())
# start_date = '2021-01-01'
# end_date = '2021-02-22'
# # df = get_data(start_date,end_date,meters)
num_stamps = (((string_2_date(end_date)-string_2_date(start_date)).days)+1)*480
num_stamps = num_stamps-110
# cols = ['Meter','start_date','end_date','num_timestamps','missing_num_stamps','missing_data_perc','power_cut_num_stamps','daily_avg_power_cut_hrs','data_swap_num_days','sum_kWh','daily_average_median_V','8-12','12-20','20-24','24']
cols = ['Meter','start_date','end_date','num_timestamps','missing_num_stamps','missing_data_perc','power_cut_num_stamps','daily_avg_power_cut_hrs','data_swap_num_days','sum_kWh','daily_average_median_V']

In [10]:
csv = df

In [11]:
num_stamps

65170

In [12]:
csv.iloc[0]

Device Sr No                SMR01-5218-0019
x_Timestamp             2021-06-27 23:57:00
t_kWh                                 0.016
t_kVAh                                 0.02
z_Avg Voltage (Volt)                 217.79
z_Max Voltage (Volt)                 219.77
z_Min Voltage (Volt)                 214.65
z_Avg Current (Amp)                    1.77
z_Max Current (Amp)                    1.79
z_Min Current (Amp)                    1.77
y_Freq (Hz)                           50.02
y_PF Total                             0.88
meter                                  BR04
Name: 0, dtype: object

In [13]:
csv

,Device Sr No,x_Timestamp,t_kWh,t_kVAh,z_Avg Voltage (Volt),z_Max Voltage (Volt),z_Min Voltage (Volt),z_Avg Current (Amp),z_Max Current (Amp),z_Min Current (Amp),y_Freq (Hz),y_PF Total,meter
0,SMR01-5218-0019,2021-06-27 23:57:00,0.016,0.020,217.79,219.77,214.65,1.77,1.79,1.77,50.02,0.88,BR04
1,SMR01-5218-0019,2021-06-27 23:54:00,0.018,0.019,215.11,215.80,214.07,1.80,1.86,1.77,50.02,0.88,BR04
2,SMR01-5218-0019,2021-06-27 23:51:00,0.030,0.032,217.81,219.30,215.62,2.94,4.21,1.81,50.00,0.93,BR04
3,SMR01-5218-0019,2021-06-27 23:48:00,0.045,0.048,218.09,219.09,216.96,4.39,4.55,4.12,50.00,0.94,BR04
4,SMR01-5218-0019,2021-06-27 23:45:00,0.034,0.036,218.36,218.98,217.17,3.31,4.40,2.58,49.98,0.94,BR04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64205,SMR01-3719-0003,2021-03-01 05:42:00,0.011,0.012,263.91,264.32,263.67,0.97,0.97,0.95,50.08,0.87,BR51
64206,SMR01-3719-0003,2021-03-01 05:39:00,0.010,0.013,263.23,264.17,262.19,0.96,0.99,0.95,50.05,0.87,BR51
64207,SMR01-3719-0003,2021-03-01 05:36:00,0.011,0.012,261.44,262.08,260.82,0.96,0.98,0.95,50.02,0.87,BR51
64208,SMR01-3719-0003,2021-03-01 05:33:00,0.012,0.014,260.96,261.50,260.35,1.15,1.62,0.95,50.00,0.81,BR51


In [14]:
# get records for all meters
record = {k: [] for k in cols}
for meter in meters:
    meter_df = csv.loc[csv['meter']==meter]
    
#     if meter == 'MH04':
#         meter = 'MH38'
    record['Meter'].append(meter)
    if meter_df.shape[0]>0:
        start_date_n = meter_df.iloc[-1]['x_Timestamp'].split(' ')[0]
        end_date_n = meter_df.iloc[0]['x_Timestamp'].split(' ')[0]
        record['start_date'].append(meter_df.iloc[-1]['x_Timestamp'].split(' ')[0])
        record['end_date'].append(meter_df.iloc[0]['x_Timestamp'].split(' ')[0])
        days = ((string_2_date(end_date_n) - string_2_date(start_date_n)).days)+1
        
    else:
        record['start_date'].append(np.nan)
        record['end_date'].append(np.nan)
        days = 1
    
    record['num_timestamps'].append(meter_df.shape[0])
    missing = num_stamps-meter_df.shape[0]
    record['missing_num_stamps'].append(missing)
    record['missing_data_perc'].append((missing/num_stamps)*100)
    pc = meter_df.loc[meter_df['z_Avg Voltage (Volt)']==0]
    power_cut_stamps = pc.shape[0]
    record['power_cut_num_stamps'].append(power_cut_stamps)
    record['daily_avg_power_cut_hrs'].append(power_cut_stamps/(20*days))
    
    meter_df = meter_df.set_index(pd.to_datetime(meter_df['x_Timestamp']))
    
#     swap_num,d = swap_num_days(meter_df)
    swap_num = swap_num_days(meter_df)
    record['data_swap_num_days'].append(swap_num)
    
    
#     record['8-12'].append(d['8-12'])
#     record['12-20'].append(d['12-20'])
#     record['20-24'].append(d['20-24'])
#     record['24'].append(d['24'])
    
    record['sum_kWh'].append(meter_df['t_kWh'].sum())
    v_not_zero = meter_df.loc[meter_df['z_Avg Voltage (Volt)']!=0]
    record['daily_average_median_V'].append(v_not_zero['z_Avg Voltage (Volt)'].median())

In [15]:
# check the dict

for k in record.keys():
    print(k)
    print(len(record[k]))

Meter
30
start_date
30
end_date
30
num_timestamps
30
missing_num_stamps
30
missing_data_perc
30
power_cut_num_stamps
30
daily_avg_power_cut_hrs
30
data_swap_num_days
30
sum_kWh
30
daily_average_median_V
30


In [16]:
summary = pd.DataFrame(record)

In [17]:
summary

,Meter,start_date,end_date,num_timestamps,missing_num_stamps,missing_data_perc,power_cut_num_stamps,daily_avg_power_cut_hrs,data_swap_num_days,sum_kWh,daily_average_median_V
0,BR04,2021-03-01,2021-06-27,56050,9120,13.994169,2472,1.038655,"(0, {'8-12': 4, '12-20': 0, '20-24': 0, '24': 0})",1230.714,239.280
1,BR05,2021-03-01,2021-05-20,27730,37440,57.449747,721,0.445062,"(0, {'8-12': 1, '12-20': 0, '20-24': 0, '24': 0})",181.940,245.320
2,BR06,2021-03-01,2021-07-13,64210,960,1.473070,2107,0.780370,"(0, {'8-12': 2, '12-20': 0, '20-24': 0, '24': 0})",2550.293,240.820
3,BR08,NaN,NaN,0,65170,100.000000,0,0.000000,"(0, {'8-12': 0, '12-20': 0, '20-24': 0, '24': 0})",0.000,NaN
4,BR11,2021-03-01,2021-07-13,64210,960,1.473070,2069,0.766296,"(0, {'8-12': 1, '12-20': 0, '20-24': 0, '24': 0})",986.019,245.640
5,BR13,2021-03-02,2021-07-13,46080,19090,29.292619,1514,0.564925,"(0, {'8-12': 0, '12-20': 0, '20-24': 0, '24': 0})",233.379,242.800
6,BR15,2021-03-01,2021-07-13,63730,1440,2.209606,1952,0.722963,"(0, {'8-12': 0, '12-20': 0, '20-24': 0, '24': 0})",1114.575,246.530
7,BR16,2021-03-01,2021-06-27,50770,14400,22.096056,2237,0.939916,"(0, {'8-12': 0, '12-20': 0, '20-24': 1, '24': 0})",145.760,232.570
8,BR19,2021-03-01,2021-07-13,64690,480,0.736535,3535,1.309259,"(0, {'8-12': 0, '12-20': 1, '20-24': 1, '24': 0})",1736.285,235.110
9,BR22,2021-03-01,2021-07-13,64210,960,1.473070,3507,1.298889,"(0, {'8-12': 0, '12-20': 1, '20-24': 1, '24': 0})",1907.484,239.490


In [18]:
summary.to_csv('instant_data_quality/'+start_date + ' -- '+ end_date + 'BR data quality.csv')

In [21]:
df.to_csv('data download/'+start_date + ' -- '+ end_date + 'BR data.csv')

In [ ]:
# Part 2

In [128]:
# data for one year
folder = 'D:\CEEW\Smart Meter Analysis\Data Acquisition and Preprocessing\data preprocess\data2'
files = os.listdir(folder)
df_list = []
for f in files:
    df = pd.read_csv(folder+ '/' + f)
    df_list.append(df.iloc[::-1])
df =pd.concat([df_list[i] for i in range(len(df_list))],axis=0)